In [1]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable


**Importing needed models**

In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model

**Determine the dataset path**

In [2]:
dataset = 'All data'
model_save_path = 'The saved model/model.keras'


**Determine the number of classes**

In [3]:
NUM_CLASSES = 6

**Define the load date function**

In [4]:
def load_dataset(dataset_path):
    x = []
    y = []
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Loop through dataset directory
    for exercise_folder in os.listdir(dataset_path):
        exercise_label = exercise_folder
        exercise_folder_path = os.path.join(dataset_path, exercise_folder)  #/content/Data1/crunches

        # Loop through video files in exercise folder
        for video_file in os.listdir(exercise_folder_path):
            video_path = os.path.join(exercise_folder_path, video_file)
            cap = cv2.VideoCapture(video_path)
            while cap.isOpened():
                success, image = cap.read()
                if not success:
                    break
                # Process image using MediaPipe Pose Detection
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                result = pose.process(image_rgb)
                if result.pose_landmarks:
                    # Extract pose landmarks
                    landmarks = [[lm.x, lm.y] for lm in result.pose_landmarks.landmark]
                    x.append(landmarks)
                    y.append(exercise_label)
            cap.release()
    return np.array(x), np.array(y)

**Call the load data function**

In [5]:
x, y = load_dataset(dataset)


In [6]:
print(f"Type of X: {type(x)}")
print(f"Type of y: {type(y)}")

Type of X: <class 'numpy.ndarray'>
Type of y: <class 'numpy.ndarray'>


In [7]:
m = 0
for i in y:
  if (i == 'squat true'):
    m+=1
print(m)

5413


In [8]:
print(y)

['crunches false' 'crunches false' 'crunches false' ... 'squat true'
 'squat true' 'squat true']


**Change the label's names**

In [9]:
for i in range(y.size):
  if y[i] == 'crunches false':
    y[i] = 0
  elif y[i] == 'crunches true':
    y[i] = 1
  elif y[i] == 'push_up false':
    y[i] = 2
  elif y[i] == 'push_up true':
    y[i] = 3
  elif y[i] == 'squat false':
    y[i] = 4
  elif y[i] == 'squat true':
    y[i] = 5
  else:
    y[i] = -1

In [10]:
print(y)

['0' '0' '0' ... '5' '5' '5']


**Making sure that the x and y is np.float32**

In [11]:
if x.dtype != np.float32:
    x = x.astype(np.float32)
if y.dtype != np.float32:
    y = y.astype(np.float32)

**Divide the dataset into train data and test data**

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

**Building the model**

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

**Defining the call back function**

In [14]:

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

**Compile the model**

In [15]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

**Training the model**

In [16]:
history = model.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=64,
    validation_split=0.2,
    callbacks=[cp_callback, es_callback]
    )


Epoch 1/150
349/359 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5265 - loss: 1.1282
Epoch 1: saving model to The saved model/model.keras
359/359 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.5298 - loss: 1.1188 - val_accuracy: 0.7385 - val_loss: 0.5778
Epoch 2/150
351/359 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7447 - loss: 0.5301
Epoch 2: saving model to The saved model/model.keras
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7452 - loss: 0.5292 - val_accuracy: 0.8114 - val_loss: 0.3971
Epoch 3/150
355/359 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8018 - loss: 0.4202
Epoch 3: saving model to The saved model/model.keras
359/359 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8020 - loss: 0.4201 - val_accuracy: 0.8522 - val_loss: 0.3283
Epoch 4/150
355/359 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8342 - loss: 0.3558
Epoch 4: saving model to The saved model/model.keras
359/359 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8342 - loss: 0.3556 - val_accuracy

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 31, 64)         │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 15, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 13, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 670,676 (2.56 MB)

 Trainable params: 223,558 (873.27 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 447,118 (1.71 MB)

**Evaluate data**

In [18]:
loss, accuracy = model.evaluate(X_test, y_test)


225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9827 - loss: 0.2048


**Load the model**

In [19]:
model = tf.keras.models.load_model(model_save_path)


In [20]:
print(model.input_shape)

(None, 33, 2)


**Test the model by videos**

In [38]:
video_path = 'v1testR.mp4'
cap = cv2.VideoCapture(video_path)

In [39]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [41]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with MediaPipe Pose Detection
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    # Check if pose landmarks are detected
    if result.pose_landmarks:
        # Extract pose landmarks
        landmarks = [[lm.x, lm.y] for lm in result.pose_landmarks.landmark]

        # Preprocess landmarks (reshape, convert to numpy array, etc.)
        # Example:
        landmarks_array = np.array(landmarks, dtype=np.float32)
        landmarks_array = landmarks_array[np.newaxis, ...]  # Add batch dimension

        # Use the model to make predictions
        predictions = model.predict(landmarks_array)

        # Example: Print the predicted class
        if (np.argmax(predictions) == 0):
          print("Predicted Class: crunches false")
        elif (np.argmax(predictions) == 1):
          print("Predicted Class: crunches true")
        elif (np.argmax(predictions) == 2):
          print("Predicted Class: push up false")
        elif (np.argmax(predictions) == 3):
          print("Predicted Class: push up true")
        elif (np.argmax(predictions) == 4):
          print("Predicted Class: squat false")
        elif (np.argmax(predictions) == 5):
          print("Predicted Class: squat true")
        else:
          print("nothing")

    # Display the frame
    cv2.imshow("output",frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
Predicted Class: push up false
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/